# Abstract Collider Prompts

This notebook first explains the datastructure used in (Rehder and Waldmann, 2017)
and then demonstrates how to **construct the RW17 domain components** using helper functions 
from `dataset_creation`, and **convert them into a structured DataFrame**.

We will:
1. **Create a domain dictionary** using `create_domain_dict`
2. **Expand it into a DataFrame** using `expand_domain_to_dataframe`
3. **Add inference tasks** to extend the dataset
4. **Generate verbalized prompts** for human evaluation


This step will:

Load and examine rw_17_domain_components (the predefined dataset components).
Break down its structure to understand:
Domain dictionary (specification of causal variables).
Graph structure (causal relationships).
Inference tasks (reasoning scenarios).
Explain how these elements combine to generate prompts for LLMs.


In [1]:
import os
import sys
import pprint
import pandas as pd

# Ensure Python finds the `src` directory
sys.path.append(os.path.abspath("../../src"))

# Import everything defined in `__all__`
from causalalign.dataset_creation import (
    rw_17_domain_components,
    graph_structures,
    inference_tasks_rw17,
    generate_prompt_dataframe,
    expand_domain_to_dataframe,
    expand_df_by_task_queries,
    create_domain_dict,
    verbalize_domain_intro,
    verbalize_causal_mechanism,
    verbalize_inference_task,
    append_dfs,
)

print("Dataset creation module imported successfully!")


Dataset creation module imported successfully!


# 1. Understanding the RW17 Dataset Structure

Before generating new datasets, we need to understand the structure of **RW17 domain components**.

## 🔹 What Is RW17?
RW17 presented humans with causal inference tasks and asked for their likelihood judgements. Each causal inference task was presented on four subsequent screens.
In the following, I will describe how I translated the experimental materials used by RW17 into nested dictionaries, which serve as the backbone to algorithmically  generate  the materials in RW17 in *textual form* such that we can prompt and compare LLMs' causal judgements. The following notebook explains how to algoritmically re-create the the textual form used by RW17 and also, how to easily create new prompts.

RW17 used 3 different knowledge domains, in particular economy, sociology, and weather, in which the inference tasks were thematically embedded.
Each domain specifies:
- **Variables (`C1`, `C2`, `E`)**: The causal variables / graph nodes, e.g., C1: interest rates
- **Variable Sense depending on binary values 0 or 1 for (`C1`, `C2`, `E`)**: e.g. for C1=1: *high* interest rates
- ** Counterbalance-dependent Sense Assignments (`p/m`)**: How we represent conditions in counterbalanced ways (optional, but used in RW17). Essentially, this flips the senses of what it means for the variable to be on (1) or off (0).

The verbalization of the prompt depends on the domain and:
- **Causal Mechanisms**: How the variables influence each other (e.g., specified by collider, graph or chain graph)
- **Inference Tasks**: The reasoning problems we ask an LLM to solve specified by 

By combining these components, we **generate structured natural language prompts** that can be used for causal reasoning tasks in an LLMs.


##  How Graph Structures Define Causal Mechanisms

A **graph structure** specifies how causal variables (`C1`, `C2`, `E`) relate to each other.

### Example Graph Structures:
1. **Collider** (`C1 → E ← C2`)
   - `C1` and `C2` both cause `E`.

2. **Fork** (`C1 ← E → C2`)
   - `E` causes both `C1` and `C2`.

3. **Chain** (`C1 → C2 → E`)
   - `C1` causes `C2`, which then affects `E`.

Let's look at the graph structures dictionary that is already pre-defined in ``src/causalalign/dataset_creation/constants.py``


In [2]:
# Pretty-print available graph structures
pprint.pprint(graph_structures)


{'chain': {'causal_template': '{c1_sense} {c1_name} causes {c2_sense} '
                              '{c2_name}. And {c2_sense} {c2_name} causes '
                              '{e_sense} {e_name}.',
           'description': 'C1→C2→E'},
 'collider': {'causal_template': '{c1_sense} {c1_name} causes {e_sense} '
                                 '{e_name}. Also, {c2_sense} {c2_name} causes '
                                 '{e_sense} {e_name}.',
              'description': 'C1→E←C2'},
 'fork': {'causal_template': '{c1_sense} {c1_name} causes {e_sense} {e_name}. '
                             'Also, {c1_sense} {c1_name} causes {c2_sense} '
                             '{c2_name}.',
          'description': 'E←C1→C2'}}


##  How Inference Tasks Define the Final Prompt

Inference tasks specify **what the LLM needs to predict** given certain observations.

For example:
- `"a": {"query_node": "Ci", "observation": "Cj=1", "query": "Ci=?"}`
  - **Ask:** Given that `Cj=1`, what is the likely value of `Ci`?
  - This corresponds to **a causal reasoning question**.

Inference tasks work together with **domain dictionaries** and **graph structures** to create **verbalized prompts**.

### 🔗 How It All Connects:
1. **Domain Dictionary** → Specifies **variables and values**.
2. **Graph Structure** → Defines **causal relationships**.
3. **Inference Tasks** → Frame **the reasoning problem**.
4. **Prompt Verbalization** → Converts this into **natural language for LLMs**.



In [3]:
pprint.pprint(inference_tasks_rw17)

{'a': {'observation': 'E=1, Cj=1',
       'query': 'p(Ci=1|E=1, Cj=1)',
       'query_node': 'Ci=1'},
 'b': {'observation': 'E=1', 'query': 'p(Ci=1|E=1)', 'query_node': 'Ci=1'},
 'c': {'observation': 'E=1, Cj=0',
       'query': 'p(Ci=1|E=1, Cj=0)',
       'query_node': 'Ci=1'},
 'd': {'observation': 'Cj=1', 'query': 'p(Ci=1|Cj=1)', 'query_node': 'Ci=1'},
 'e': {'observation': 'Cj=0', 'query': 'p(Ci=1|Cj=0)', 'query_node': 'Ci=1'},
 'f': {'observation': 'E=0, Cj=1',
       'query': 'p(Ci=1|E=0, Cj=1)',
       'query_node': 'Ci=1'},
 'g': {'observation': 'E=0', 'query': 'p(Ci=1|E=0)', 'query_node': 'Ci=1'},
 'h': {'observation': 'E=0, Cj=0',
       'query': 'p(Ci=1|E=0, Cj=0)',
       'query_node': 'Ci=1'},
 'i': {'observation': 'Ci=0, Cj=0',
       'query': 'p(E=1|Ci=0, Cj=0)',
       'query_node': 'E=1'},
 'j': {'observation': 'Ci=0, Cj=1',
       'query': 'p(E=1|Ci=0, Cj=1)',
       'query_node': 'E=1'},
 'k': {'observation': 'Ci=1, Cj=1',
       'query': 'p(E=1|Ci=1, Cj=1)',
       

In [4]:
scocio_dict_const = rw_17_domain_components["sociology"]
scocio_dict_const

{'domain_name': 'sociology',
 'variables': {'C1': {'C1_name': 'urbanization',
   'C1_detailed': 'Urbanization is the degree to which the members of a society live in urban environments (i.e., cities) versus rural environments.',
   'p_value': {'1': 'high', '0': 'normal'},
   'm_value': {'1': 'low', '0': 'normal'},
   'explanations': {'p_p': 'Big cities provide many opportunities for financial and social improvement.',
    'p_m': 'In big cities many people are competing for the same high-status jobs and occupations.',
    'm_p': 'People in rural areas are rarely career oriented, and so take time off from working and switch frequently between different "temp" jobs.',
    'm_m': 'The low density of people prevents the dynamic economic expansion needed for people to get ahead.'}},
  'C2': {'C2_name': 'interest in religion',
   'C2_detailed': 'Interest in religion is the degree to which the members of a society show a curiosity in religion issues or participate in organized religions.',
   

In [5]:
const_df_test_socio = generate_prompt_dataframe(
    domain_dict=scocio_dict_const,
    inference_tasks=inference_tasks_rw17,
    graph_type="collider",
    graph_structures=graph_structures,
    counterbalance_enabled=True,
)


Now, let's explore how these components are combined by re-creating the prompts used in RW17 starting with re-creating the dictionaries that are stored in ``constants.py``!


# Step 1: Create Domain Dictionray:


In [6]:
# # Create individual domains
# economy_domain_dict = create_domain_dict(
#     domain="economy",
#     introduction="Economists seek to describe and predict the regular patterns of economic fluctuation. To do this, they study some important variables or attributes of economies. They also study how these attributes are responsible for producing or causing one another.",
#     C1_name="interest rates",
#     C1_detailed="Interest rates are the rates banks charge to loan money.",
#     C1_values={"1": "low", "0": "high"},
#     C2_name="trade deficits",
#     C2_detailed="A country's trade deficit is the difference between the value of the goods that a country imports and the value of the goods that a country exports.",
#     C2_values={"1": "small", "0": "large"},
#     E_name="retirement savings",
#     E_detailed="Retirement savings is the money people save for their retirement.",
#     E_values={"1": "high", "0": "low"},
#     counterbalance_enabled=True,
# )

# sociology_domain_dict = create_domain_dict(
#     domain="sociology",
#     introduction="Sociologists seek to describe and predict the regular patterns of societal interactions. To do this, they study some important variables or attributes of societies. They also study how these attributes are responsible for producing or causing one another.",
#     C1_name="urbanization",
#     C1_detailed="Urbanization is the degree to which the members of a society live in urban environments (i.e., cities) versus rural environments.",
#     C1_values={"1": "high", "0": "low"},
#     C2_name="interest in religion",
#     C2_detailed="Interest in religion is the degree to which the members of a society show a curiosity in religion issues or participate in organized religions.",
#     C2_values={"1": "low", "0": "high"},
#     E_name="socio-economic mobility",
#     E_detailed="Socioeconomic mobility is the degree to which the members of a society are able to improve their social and economic status.",
#     E_values={"1": "high", "0": "low"},
#     counterbalance_enabled=True,
# )

# weather_domain_dict = create_domain_dict(
#     domain="weather",
#     introduction="Meteorologists seek to describe and predict the regular patterns that govern weather systems. To do this, they study some important variables or attributes of weather systems. They also study how these attributes are responsible for producing or causing one another.",
#     C1_name="ozone levels",
#     C1_detailed="Ozone is a gaseous allotrope of oxygen (O3) and is formed by exposure to UV radiation.",
#     C1_values={"1": "high", "0": "low"},
#     C2_name="air pressure",
#     C2_detailed="Air pressure is force exerted due to concentrations of air molecules.",
#     C2_values={"1": "low", "0": "high"},
#     E_name="humidity",
#     E_detailed="Humidity is the degree to which the atmosphere contains water molecules.",
#     E_values={"1": "high", "0": "low"},
#     counterbalance_enabled=True,
# )


abc_domain_dict = create_domain_dict(
    domain="abc",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="A",
    C1_detailed="",
    C1_values={"1": "high", "0": "low"},
    C2_name="B",
    C2_detailed="",
    C2_values={"1": "small", "0": "large"},
    E_name="C",
    E_detailed="",
    E_values={"1": "long", "0": "short"},
    counterbalance_enabled=True,
)

xyz_domain_dict = create_domain_dict(
    domain="xyz",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="X",
    C1_detailed="",
    C1_values={"1": "high", "0": "low"},
    C2_name="Y",
    C2_detailed="",
    C2_values={"1": "small", "0": "large"},
    E_name="Z",
    E_detailed="",
    E_values={"1": "long", "0": "short"},
    counterbalance_enabled=True,
)

# D, Y, X → In causal inference, where D is the treatment, Y is the outcome, and X represents covariates.
dyx_domain_dict = create_domain_dict(
    domain="dyx",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="D",
    C1_detailed="",
    C1_values={"1": "strong", "0": "weak"},
    C2_name="X",
    C2_detailed="",
    C2_values={"1": "low", "0": "high"},
    E_name="Y",
    E_detailed="",
    E_values={"1": "long", "0": "short"},
    counterbalance_enabled=True,
)


# q1, q2, q3 → Used in dynamical systems or quantum mechanics.
q123_domain_dict = create_domain_dict(
    domain="q123",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="q1",
    C1_detailed="",
    C1_values={"1": "high-energy", "0": "low-engergy"},
    C2_name="q2",
    C2_detailed="",
    C2_values={"1": "prolonged", "0": "shortened"},
    E_name="q3",
    E_detailed="",
    E_values={"1": "extended", "0": "short"},
    counterbalance_enabled=True,
)

dyx_medicine_domain_dict = create_domain_dict(
    domain="dyx_medicine",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="D",
    C1_detailed="",  # Represents the treatment level or dosage: e.g. Dose of a drug
    C1_values={
        "1": "high",
        "0": "low",
    },  # or {"1": "intensive", "0": "mild"} for therapy
    C2_name="X",
    C2_detailed="",  # Represents patient-related covariates that may affect the outcome. e.g. risk
    C2_values={
        "1": "high",
        "0": "low",
    },  # or {"1": "young", "0": "elderly"} if age is the covariate
    E_name="Y",
    E_detailed="",  # Represents the observed health outcome., e.g. duration of recovery
    E_values={
        "1": "long",
        "0": "short",
    },  # or {"1": "severe", "0": "mild"} for disease severity
    counterbalance_enabled=True,
)

dyx_economics_domain_dict = create_domain_dict(
    domain="dyx_economics",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="D",
    C1_detailed="",  # Represents the strength of an economic policy or intervention: e.g. Tax policy strength
    C1_values={
        "1": "strong",
        "0": "weak",
    },  # or {"1": "high", "0": "low"} for subsidies
    C2_name="X",
    C2_detailed="",  # Represents economic conditions that may influence the policy's effectiveness: e.g. market stability
    C2_values={
        "1": "unstable",
        "0": "stable",
    },  # or {"1": "low", "0": "high"} for income level
    E_name="Y",
    E_detailed="",  # Represents the observed economic impact of the policy: e.g. economic growth
    E_values={
        "1": "high",
        "0": "low",
    },  # or {"1": "long", "0": "short"} for unemployment duration
    counterbalance_enabled=True,
)

xyz_psychology_domain_dict = create_domain_dict(
    domain="xyz_psychology",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="X",
    C1_detailed="",  # Represents the intensity of the intervention or stimulus exposure: e.g. stress level
    C1_values={
        "1": "high",
        "0": "low",
    },  # or {"1": "intense", "0": "mild"} for cognitive training
    C2_name="Y",
    C2_detailed="",  # Represents individual differences that may influence response to the intervention: e.g. anxiety level
    C2_values={
        "1": "high",
        "0": "low",
    },  # or {"1": "experienced", "0": "novice"} for prior knowledge
    E_name="Z",
    E_detailed="",  # Represents the measured cognitive or behavioral outcome: e.g. reaction time
    E_values={
        "1": "slow",
        "0": "fast",
    },  # or {"1": "high", "0": "low"} for learning retention
    counterbalance_enabled=True,
)

# uncommon single letters
gft_domain_dict = create_domain_dict(
    domain="gft",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="G",
    C1_detailed="",
    C1_values={"1": "high", "0": "low"},
    C2_name="F",
    C2_detailed="",
    C2_values={"1": "small", "0": "large"},
    E_name="T",
    E_detailed="",
    E_values={"1": "long", "0": "short"},
    counterbalance_enabled=True,
)


abstract_domain_dict = create_domain_dict(
    domain="xqt",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="fwp",
    C1_detailed="",
    C1_values={"1": "high", "0": "low"},
    C2_name="blg",
    C2_detailed="",
    C2_values={"1": "small", "0": "large"},
    E_name="drk",
    E_detailed="",
    E_values={"1": "large", "0": "small"},
    counterbalance_enabled=True,
)


very_abstract_domain_dict = create_domain_dict(
    domain="zorpentix",
    introduction="In the following, you will be presented with causal inference tasks based on a causal mechanism consisting of 3 variables that works like this:",
    C1_name="flarnox",
    C1_detailed="",
    C1_values={"1": "present", "0": "absent"},
    C2_name="drimbex",
    C2_detailed="",
    C2_values={"1": "present", "0": "absent"},
    E_name="quorvex",
    E_detailed=".",
    E_values={"1": "high", "0": "low"},
    counterbalance_enabled=True,
)


This should re-create our rw17 dictionary. Let's verify this.

In [7]:
pprint.pprint(abc_domain_dict)

{'domain_name': 'abc',
 'introduction': 'In the following, you will be presented with causal '
                 'inference tasks based on a causal mechanism consisting of 3 '
                 'variables that works like this:',
 'variables': {'C1': {'C1_detailed': '',
                      'C1_name': 'A',
                      'm_value': {'0': 'high', '1': 'low'},
                      'p_value': {'0': 'low', '1': 'high'}},
               'C2': {'C2_detailed': '',
                      'C2_name': 'B',
                      'm_value': {'0': 'small', '1': 'large'},
                      'p_value': {'0': 'large', '1': 'small'}},
               'E': {'E_detailed': '',
                     'E_name': 'C',
                     'm_value': {'0': 'long', '1': 'short'},
                     'p_value': {'0': 'short', '1': 'long'}}}}


## Step 2: Expand the Dictionaries into Prompt Components in Dataframe

In [8]:
# create a dataframe for each domain and then append them together

abc_df = expand_domain_to_dataframe(
    abc_domain_dict,
)
xyz_domain_df = expand_domain_to_dataframe(
    xyz_domain_dict,
)
abstract_domain_df = expand_domain_to_dataframe(
    abstract_domain_dict,
)
very_abstract_df = expand_domain_to_dataframe(
    very_abstract_domain_dict,
)


### Let's look at the dataframe:

In [9]:
print(
    f"Each domain dataframe now has {len(abc_df)} rows  \n and the following columns: \n {abc_df.columns}."
)
abc_df

Each domain dataframe now has 8 rows  
 and the following columns: 
 Index(['domain', 'C1', 'C1_values', 'C1_cntbl', 'C1_sense', 'C1_detailed',
       'C2', 'C2_values', 'C2_cntbl', 'C2_sense', 'C2_detailed', 'E',
       'E_values', 'E_cntbl', 'E_sense', 'E_detailed', 'cntbl_cond'],
      dtype='object').


,domain,C1,C1_values,C1_cntbl,C1_sense,C1_detailed,C2,C2_values,C2_cntbl,C2_sense,C2_detailed,E,E_values,E_cntbl,E_sense,E_detailed,cntbl_cond
0,abc,A,1,p,high,,B,1,p,small,,C,1,p,long,,ppp
1,abc,A,1,p,high,,B,1,p,small,,C,1,m,short,,ppm
2,abc,A,1,p,high,,B,1,m,large,,C,1,p,long,,pmp
3,abc,A,1,p,high,,B,1,m,large,,C,1,m,short,,pmm
4,abc,A,1,m,low,,B,1,p,small,,C,1,p,long,,mpp
5,abc,A,1,m,low,,B,1,p,small,,C,1,m,short,,mpm
6,abc,A,1,m,low,,B,1,m,large,,C,1,p,long,,mmp
7,abc,A,1,m,low,,B,1,m,large,,C,1,m,short,,mmm


## Creating the entire prompt for each condition and domain:

- Step 1:  for each domain dictionary, create the domain dataframe and then the verbalizations
- Step 2: append all complete dataframes
- Step 3: _optionally_ subset for counterbalance conditions of interest
- Step 4: _optionally_ merge with human data



## Custom Response prompt instructions:


Please provide your response in the following strict XML format without any additional text or explanation:
```
<response>
    <likelihood>YOUR_NUMERIC_RESPONSE_HERE</likelihood>
    <confidence>YOUR_CONFIDENCE_SCORE_HERE</confidence>
</response>

```

Only replace YOUR_NUMERIC_RESPONSE_HERE with a number between 0 (unlikely) and 100 (very likely), and YOUR_CONFIDENCE_SCORE_HERE with a number between 0 (very uncertain) and 100 (very certain). Do not include any explanations or text outside the XML format.


In [10]:
xml_format_numeric_certainty = "<response><likelihood>YOUR_NUMERIC_RESPONSE_HERE</likelihood><confidence>YOUR_CONFIDENCE_SCORE_HERE</confidence></response>"
xml_explanation_numeric_certainty = "Only replace YOUR_NUMERIC_RESPONSE_HERE with a number between 0 (unlikely) and 100 (very likely), and YOUR_CONFIDENCE_SCORE_HERE with a number between 0 (very uncertain) and 100 (very certain). DO NOT include any other information or text in your response and DO NOT use any special characters or symbols like quotation marks around your text output. Only return the XML inline as raw text"
prompt_type_xml_numeric_certainty = (
    "Return your response as raw text in one single line using this exact XML format: "
    + xml_format_numeric_certainty
    + " "
    + "DO NOT use Markdown, code blocks, or any additional formatting."
    # "Return your response in a single line, without line breaks within the following XML format. Do NOT use Markdown, code blocks, or any additional formatting. DO NOT add quotation marks around the response. DO NOT provide any additional information. Output must be in the exact XML format below, inline "
    # "Please provide your response in the following strict XML format. Provide your response in a single line. DO NOT provide any additional information outside the  formatting like line breaks and ensure the XML is returned as raw text without any code markers e.g., ```xml or quotation marks: "
    # + xml_format_numeric_certainty
    + " "
    + xml_explanation_numeric_certainty
)


##### Next, we'll call `generate_prompt_dataframe()` for each domain dictionary. 
We'll start with one domain dictionary and after that, we'll loop over the remaining domain dictionaries
and append the resulting dataframes.

In [11]:
# create an empty dataframe to append the domain dataframes to
abstract_complete_df = generate_prompt_dataframe(
    domain_dict=abc_domain_dict,
    inference_tasks=inference_tasks_rw17,
    graph_type="collider",
    graph_structures=graph_structures,
    counterbalance_enabled=True,
    prompt_category="numeric-certainty",
    prompt_type=prompt_type_xml_numeric_certainty,
)

# list of remaining domain dictionaries
domain_dicts_xs = [
    q123_domain_dict,
    dyx_economics_domain_dict,
    xyz_psychology_domain_dict,
    dyx_medicine_domain_dict,
    dyx_domain_dict,
    xyz_domain_dict,
    gft_domain_dict,
    abstract_domain_dict,
    very_abstract_domain_dict,
]


# we'll start with the completed economy dataframe and append the other domain dataframes to it
new_over_complete_df = (
    abstract_complete_df.copy()  # over complete means, it has all 8 possible counterbalance conditions.
    # we'll subset later for the 4 counterbalance conditions that were used
)


##### now loop over remaining domain dicts and append them to the one created above

In [12]:
# loop over remaining domain dictionaries and append the
# resulting dataframes to the first dataframe created in the cell above

for dict in domain_dicts_xs:
    # for dict in rw_17_domain_components.values():
    df = generate_prompt_dataframe(
        domain_dict=dict,
        inference_tasks=inference_tasks_rw17,
        graph_type="collider",
        graph_structures=graph_structures,
        counterbalance_enabled=True,
        prompt_category="numeric-certainty",
        prompt_type=prompt_type_xml_numeric_certainty,
    )
    # append the new dataframe to the complete dataframe
    new_over_complete_df = append_dfs(new_over_complete_df, df)

In [13]:
abstract_complete_df["cntbl_cond"].unique()

array(['ppp', 'ppm', 'pmp', 'pmm', 'mpp', 'mpm', 'mmp', 'mmm'],
      dtype=object)

In [14]:
new_over_complete_df["cntbl_cond"].unique()

array(['ppp', 'ppm', 'pmp', 'pmm', 'mpp', 'mpm', 'mmp', 'mmm'],
      dtype=object)

### Subsetting for the Counterbalance conditions used in RW17
- and adding unique id.

In [15]:
# add a unique id to each row
new_over_complete_df["id"] = range(1, len(new_over_complete_df) + 1)

# subset for the counterbalance conditions used in the RW17 study
# select_contbl_cond_xs = ["ppp", "pmm", "mmp", "mpm"]
select_contbl_cond_xs = ["ppp", "pmm", "mmp", "mpm"]

In [16]:
new_complete_df = new_over_complete_df[
    new_over_complete_df["cntbl_cond"].isin(select_contbl_cond_xs)
]

# save new_complete_df to a csv file#
prompt_category = new_complete_df["prompt_category"].unique()[0]
graph_type = new_complete_df["graph"].unique()[0]
new_complete_df.to_csv(
    f"../datasets/abstract_collider_prompts/abc_abstract_{prompt_category}_LLM_prompting_{graph_type}_abstract.csv",
    index=False,
)


In [17]:
print(len(new_complete_df))
print(new_complete_df.columns)
print(new_complete_df["task"].unique())
print(new_complete_df["domain"].unique())


800
Index(['domain', 'C1', 'C1_values', 'C1_cntbl', 'C1_sense', 'C1_detailed',
       'C2', 'C2_values', 'C2_cntbl', 'C2_sense', 'C2_detailed', 'E',
       'E_values', 'E_cntbl', 'E_sense', 'E_detailed', 'cntbl_cond', 'task',
       'query_node', 'observation', 'query', 'graph', 'prompt',
       'prompt_category', 'id'],
      dtype='object')
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k']
['abc' 'q123' 'dyx_economics' 'xyz_psychology' 'dyx_medicine' 'dyx' 'xyz'
 'gft' 'xqt' 'zorpentix']


## Saving Dataframes to csv

### To prompt the the LLMs, we only need the id and prompt.
- to not confuse the LLMs, we should group them by counterbalance condition when interacting with them through the website.
- however, the default when interacting with them throught their respective APIs is that they're stateless, meaning they don't retain anything from the previous context.
    - this is why we don't have to worry about having the different counterbalance conditions contradict each other and hence confuse the LLMs


In [ ]:
# save for prompting LLMs
LLM_prompting_df = new_complete_df[
    ["id", "prompt", "prompt_category", "graph", "domain", "cntbl_cond", "task"]
]
prompt_category = LLM_prompting_df["prompt_category"].unique()[0]
graph_type = LLM_prompting_df["graph"].unique()[0]
print(prompt_category, graph_type)
LLM_prompting_df.head(
    11
).to_csv(  ## NOTE: only saving the first 11 rows for now, to check the output. DELETE THIS LINE LATER
    f"../datasets/abstract_collider_prompts/prompts_for_LLM_api/abc_{prompt_category}_LLM_prompting_{graph_type}_abstract.csv",
    index=False,
)

print("Prompts for LLMs saved successfully!")
print("prompt data file has the following columns:")
print(LLM_prompting_df.columns)

numeric-certainty collider
Prompts for LLMs saved successfully!
prompt data file has the following columns:
Index(['id', 'prompt', 'prompt_category', 'graph', 'domain', 'cntbl_cond',
       'task'],
      dtype='object')
